# APPRENTISSAGE PROFOND POUR LES JEUX - TP 4

Objectif :


L'objectif de ce TP est de familiariser les étudiants avec l'implémentation de l'algorithme
Proximal Policy Optimization (PPO). A travers ce TP, les étudiants apprendront à construire
une table de politiques, à mettre à jour la valeur des états et à entraîner un agent à résoudre
le problème de transport de passagers dans l'environnement Taxi-v3.



## Exercice 1: Initialisation de l'environnement et des structures de données

- Initialiser l'environnement Taxi-v3 et afficher le nombre d'états et d'actions.
- Créer une table de politique où chaque état a une probabilité égale pour chaque action.
- Créer une table de valeurs initialisée à zéro.
- Ajouter un affichage des premières lignes de policy_table et value_table.

In [6]:
import gymnasium as gym
import numpy as np

# Initialisation de l'environnement Taxi
env = gym.make("Taxi-v3")

# Taille de l'espace d'états et d'actions
state_size = env.observation_space.n  # 500 états
action_size = env.action_space.n      # 6 actions possibles

print(f" Nombre d'états : {state_size}")
print(f" Nombre d'actions : {action_size}")

# Création d'une table de politique : probas égales pour chaque action
policy_table = np.full((state_size, action_size), 1.0 / action_size)

# Création d'une table de valeurs initialisée à zéro
value_table = np.zeros(state_size)

# Affichage des premières lignes
print("\n Extrait de la policy_table (probas par action) :")
print(policy_table[:5])

print("\n Extrait de la value_table (valeurs par état) :")
print(value_table[:5])


 Nombre d'états : 500
 Nombre d'actions : 6

 Extrait de la policy_table (probas par action) :
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

 Extrait de la value_table (valeurs par état) :
[0. 0. 0. 0. 0.]


## Exercice 2: Exploration et collecte d'épisodes
- Faire exécuter un agent aléatoire dans l'environnement pendant 20 épisodes.
- Afficher les actions exécutées et les récompenses obtenues.

In [9]:
env = gym.make("Taxi-v3")

num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    step = 0

    print(f"\n Épisode {episode + 1}")
    
    while not done:
        action = env.action_space.sample()  # Action aléatoire
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        print(f"Étape {step}: Action = {action}, Récompense = {reward}")
        total_reward += reward
        state = next_state
        step += 1

    print(f"✅ Épisode terminé en {step} étapes avec récompense totale : {total_reward}")



 Épisode 1
Étape 0: Action = 0, Récompense = -1
Étape 1: Action = 4, Récompense = -10
Étape 2: Action = 3, Récompense = -1
Étape 3: Action = 2, Récompense = -1
Étape 4: Action = 2, Récompense = -1
Étape 5: Action = 4, Récompense = -10
Étape 6: Action = 5, Récompense = -10
Étape 7: Action = 5, Récompense = -10
Étape 8: Action = 5, Récompense = -10
Étape 9: Action = 0, Récompense = -1
Étape 10: Action = 4, Récompense = -10
Étape 11: Action = 3, Récompense = -1
Étape 12: Action = 2, Récompense = -1
Étape 13: Action = 1, Récompense = -1
Étape 14: Action = 4, Récompense = -10
Étape 15: Action = 1, Récompense = -1
Étape 16: Action = 1, Récompense = -1
Étape 17: Action = 5, Récompense = -10
Étape 18: Action = 2, Récompense = -1
Étape 19: Action = 5, Récompense = -10
Étape 20: Action = 2, Récompense = -1
Étape 21: Action = 3, Récompense = -1
Étape 22: Action = 0, Récompense = -1
Étape 23: Action = 3, Récompense = -1
Étape 24: Action = 3, Récompense = -1
Étape 25: Action = 5, Récompense = -10


## Exercice 3 : Mise à jour de la politique avec PРО
L'algorithme PPO optimise la politique z0 en maximisant la fonction suivante avec un terme de clipping pour éviter des mises à jour trop brutales:

Étapes clés de la mise à jour PPO
1. Calcul des récompenses cumulées Rt (discounted rewards).
2. Calcul de l'avantage At=Rt - V(st).
3. Mise à jour de la politique avec clipping : Ajustern en respectant les contraintes de PPО.
4. Mise à jour de la fonction de valeur V(s).
- Calculer les récompenses cumulées (discounted rewards).
- Mettre à jour la fonction de valeur pour chaque état visité.
- Mettre à jour la politique avec PPO (en respectant le clip). 
- Ajouter une mise à jour de value_table[state] avec une learning rate.


In [16]:
# Initialisation de l'environnement
env = gym.make("Taxi-v3")
state_size = env.observation_space.n
action_size = env.action_space.n

# Tables
policy_table = np.full((state_size, action_size), 1.0 / action_size)
value_table = np.zeros(state_size)

# Hyperparamètres
gamma = 0.99
lr_policy = 0.1
lr_value = 0.1
clip_epsilon = 0.2
num_episodes = 1000

# Pour suivi des récompenses
episode_rewards_list = []

# --- Fonction : Récompenses cumulées ---
def compute_discounted_rewards(rewards, gamma=0.99):
    discounted = np.zeros_like(rewards, dtype=np.float32)
    cumulative = 0
    for t in reversed(range(len(rewards))):
        cumulative = rewards[t] + gamma * cumulative
        discounted[t] = cumulative
    return discounted

# --- Fonction : Mise à jour PPO ---
def update_with_ppo(states, actions, rewards,
                    value_table, policy_table,
                    gamma=0.99, lr_value=0.1, lr_policy=0.1, clip_epsilon=0.2):

    discounted_rewards = compute_discounted_rewards(rewards, gamma)

    for t in range(len(states)):
        state = states[t]
        action = actions[t]
        Gt = discounted_rewards[t]
        V_st = value_table[state]
        advantage = Gt - V_st

        # Ratio de proba
        old_prob = policy_table[state, action]
        r_theta = old_prob / (old_prob + 1e-10)
        clipped_r = np.clip(r_theta, 1 - clip_epsilon, 1 + clip_epsilon)
        policy_gradient = min(r_theta * advantage, clipped_r * advantage)

        # Mise à jour politique
        policy_table[state, action] += lr_policy * policy_gradient

        # Correction : clip + normalisation
        policy_table[state] = np.clip(policy_table[state], 1e-6, None)
        policy_table[state] = policy_table[state] / np.sum(policy_table[state])

        # Mise à jour valeur
        value_table[state] += lr_value * advantage

# --- Entraînement principal ---
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False

    episode_states = []
    episode_actions = []
    episode_rewards = []

    while not done:
        action_probs = policy_table[state]
        action = np.random.choice(action_size, p=action_probs)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)

        state = next_state

    update_with_ppo(episode_states, episode_actions, episode_rewards,
                    value_table, policy_table,
                    gamma, lr_value, lr_policy, clip_epsilon)

    total_reward = sum(episode_rewards)
    episode_rewards_list.append(total_reward)

    if (episode + 1) % 100 == 0:
        print(f" Épisode {episode + 1} - Récompense totale : {total_reward}")

# --- Résultats finaux ---
print("\n✅ Entraînement PPO terminé.")
print(" Extrait de la value_table :")
print(value_table[:10])

print("\n Extrait de la politique (probas pour les 5 premiers états) :")
for s in range(5):
    print(f"État {s} → {policy_table[s]}")


 Épisode 100 - Récompense totale : -200
 Épisode 200 - Récompense totale : -200
 Épisode 300 - Récompense totale : -200
 Épisode 400 - Récompense totale : -200
 Épisode 500 - Récompense totale : -371
 Épisode 600 - Récompense totale : -200
 Épisode 700 - Récompense totale : -200
 Épisode 800 - Récompense totale : -2000
 Épisode 900 - Récompense totale : -200
 Épisode 1000 - Récompense totale : -200

✅ Entraînement PPO terminé.
 Extrait de la value_table :
[   0.          -16.03574665  -78.77969126   -9.17431185 -253.03499997
    0.          -91.74311844 -114.33893083 -133.99994114   -9.63786854]

 Extrait de la politique (probas pour les 5 premiers états) :
État 0 → [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
État 1 → [3.74441854e-07 3.74441854e-07 9.99998128e-01 3.74441854e-07
 3.74441854e-07 3.74441854e-07]
État 2 → [1.53846408e-01 3.84613598e-01 1.07692270e-06 3.07691355e-01
 7.69236272e-02 7.69239349e-02]
État 3 → [5.24037814e-07 5.24037814e-07 5.24037814e-0

## Exercice 4: Évaluation de l'agent après entraînement
- Tester l'agent entraîné pendant 20 épisodes.
- Comparer les performances avant et après entraînement.

In [19]:
# Nombre d'épisodes pour l'évaluation
num_eval_episodes = 20

# --- Fonction pour évaluer un agent donné ---
def evaluate_agent(policy_table, mode="greedy", episodes=20):
    total_rewards = []
    
    for ep in range(episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            if mode == "greedy":
                action = np.argmax(policy_table[state])  # action max
            elif mode == "random":
                action = env.action_space.sample()       # action aléatoire
            else:
                raise ValueError("Mode inconnu : choisir 'greedy' ou 'random'")
            
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            state = next_state
        
        total_rewards.append(total_reward)
    
    return total_rewards

# --- Évaluation AVANT entraînement (agent aléatoire) ---
print(" Évaluation AVANT entraînement (agent aléatoire)")
random_rewards = evaluate_agent(policy_table, mode="random", episodes=num_eval_episodes)
print(f"Récompenses : {random_rewards}")
print(f"Récompense moyenne (aléatoire) : {np.mean(random_rewards):.2f}")

# --- Évaluation APRÈS entraînement (agent PPO) ---
print("\n Évaluation APRÈS entraînement (agent entraîné avec PPO)")
ppo_rewards = evaluate_agent(policy_table, mode="greedy", episodes=num_eval_episodes)
print(f"Récompenses : {ppo_rewards}")
print(f"Récompense moyenne (PPO) : {np.mean(ppo_rewards):.2f}")

# --- Comparaison ---
improvement = np.mean(ppo_rewards) - np.mean(random_rewards)
print(f"\n Amélioration moyenne de la récompense : {improvement:.2f}")


 Évaluation AVANT entraînement (agent aléatoire)
Récompenses : [-839, -848, -803, -695, -767, -740, -821, -740, -839, -758, -821, -794, -785, -812, -857, -713, -803, -857, -839, -740]
Récompense moyenne (aléatoire) : -793.55

 Évaluation APRÈS entraînement (agent entraîné avec PPO)
Récompenses : [-200, -200, -200, -2000, -200, -2000, -200, -200, -200, -200, -200, -200, -2000, -200, -200, -200, -200, -200, -2000, -200]
Récompense moyenne (PPO) : -560.00

 Amélioration moyenne de la récompense : 233.55
